In [6]:
import os
import pandas as pd
import json

In [7]:
%run utils/__init__.py

In [8]:
%run metrics/__init__.py

In [55]:
classification = False

In [56]:
base_folder = os.path.join(WORKSPACE_DIR,
                           'classification' if classification else 'report_generation')
results_folder = os.path.join(base_folder, 'results')
runs_folder = os.path.join(base_folder, 'runs')
run_names = set(os.listdir(results_folder)).intersection(set(os.listdir(runs_folder)))
run_names = list(run_names)
if 'debug' in run_names: run_names.remove('debug')
run_names

['0716_211601_lstm-att_lr0.0001_densenet-121',
 '0716_234501_h-lstm-att_lr0.0001_densenet-121',
 '0717_015057_h-lstm_lr0.0001_densenet-121']

In [57]:
filename = 'metrics'

suffix = 'max-words-27'
if suffix:
    filename += f'-{suffix}'
filename

'metrics-max-words-27'

In [58]:
all_results = []

for run_name in run_names:
    filepath = os.path.join(results_folder, run_name, f'{filename}.json')
    
    if not os.path.isfile(filepath):
        print('Not found for: ', run_name)
        continue

    with open(filepath, 'r') as f:
        results = json.load(f)

    all_results.append((run_name, results))

len(all_results)

Not found for:  0716_234501_h-lstm-att_lr0.0001_densenet-121
Not found for:  0717_015057_h-lstm_lr0.0001_densenet-121


1

In [59]:
def create_results_df(key):
    run_names = [run_name for run_name, _ in all_results]
    
    results_dicts = [d[key] for _, d in all_results]
    
    df = pd.DataFrame(results_dicts, index=run_names)
    return df

In [60]:
results_df_train = create_results_df('train')
results_df_val = create_results_df('val')
results_df_test = create_results_df('test')

### Report generation

In [17]:
results_df_test = results_df_test.loc[results_df_test.index.str.contains('_lstm_')]

In [61]:
results_df_val

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0716_211601_lstm-att_lr0.0001_densenet-121,8.513943,0.068213,0.434808,0.277032,0.194837,0.147231,0.263477,0.348825,0.741187


In [62]:
results_df_test

,loss,word_acc,bleu1,bleu2,bleu3,bleu4,bleu,rougeL,ciderD
0716_211601_lstm-att_lr0.0001_densenet-121,7.386976,0.078631,0.446062,0.288663,0.202485,0.149504,0.271678,0.357307,0.748227


In [19]:
print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]
print((results_df_test).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  word\_acc &  bleu1 &  bleu2 &  bleu3 &  bleu4 &  bleu &  rougeL &  ciderD \\
\midrule
0714\_122624\_lstm\_lr0.0001\_densenet-121 &      0.09 &   0.50 &   0.32 &   0.21 &   0.15 &  0.30 &    0.36 &    0.48 \\
0714\_100643\_lstm\_lr1e-05\_densenet-121  &      0.11 &   0.22 &   0.10 &   0.02 &   0.00 &  0.08 &    0.21 &    0.07 \\
\bottomrule
\end{tabular}



### Classification

In [37]:
# contains = 'covid-x'
contains = 'cxr14'

In [47]:
results_df_train = results_df_train.loc[results_df_train.index.str.contains(contains)]
results_df_val = results_df_val.loc[results_df_val.index.str.contains(contains)]
results_df_test = results_df_test.loc[results_df_test.index.str.contains(contains)]

In [48]:
results_df_train.sort_values(['acc', 'recall_covid', 'prec_covid'],
                           ascending=False).dropna(axis=1, how='all')

,loss,acc,hamming,bce,acc_Pneumonia,prec_Pneumonia,recall_Pneumonia,spec_Pneumonia,roc_auc_Pneumonia,cm_Pneumonia,...,cm_Infiltration,cm_Mass,cm_Nodule,cm_Pneumothorax,cm_Consolidation,cm_Edema,cm_Emphysema,cm_Fibrosis,cm_Pleural_Thickening,cm_Hernia
0715_232631_cxr14_densenet-121_lr1e-06_os-max4_aug-0_focal_Pneumonia,0.008517,0.999946,0.000011,0.001073,0.999989,0.999945,1.000000,0.999987,1.000000,"[[74951, 1], [0, 18288]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0715_232418_cxr14_densenet-121_lr1e-06_os-max4_aug-0_Pneumonia,0.175250,0.998715,0.000257,0.025740,0.999743,0.998689,1.000000,0.999680,1.000000,"[[74928, 24], [0, 18288]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0716_133211_cxr14_densenet-121_lr1e-06_us_aug-0_Pneumonia,11.794916,0.850771,0.076334,7.633421,0.923666,0.978272,0.866579,0.980752,0.967767,"[[4484, 88], [610, 3962]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0714_232500_cxr14_densenet-121_lr1e-06,14.382884,0.337455,0.099919,9.991925,0.985089,0.033079,0.017060,0.994930,0.652203,"[[74572, 380], [749, 13]]",...,"[[38464, 25223], [5369, 6658]]","[[65619, 6516], [2893, 686]]","[[66275, 5308], [3469, 662]]","[[69902, 3457], [2002, 353]]","[[69034, 4175], [2075, 430]]","[[73380, 1116], [1088, 130]]","[[72592, 1816], [1114, 192]]","[[74187, 420], [1078, 29]]","[[70646, 3081], [1737, 250]]","[[75505, 83], [126, 0]]"
0714_232518_cxr14_densenet-121_lr1e-06,527.095947,0.221361,0.232625,23.262542,0.952043,0.028900,0.115486,0.960548,0.681557,"[[71995, 2957], [674, 88]]",...,"[[5683, 58004], [527, 11500]]","[[46722, 25413], [1567, 2012]]","[[43583, 28000], [1792, 2339]]","[[58724, 14635], [1311, 1044]]","[[62237, 10972], [1502, 1003]]","[[71397, 3099], [892, 326]]","[[71483, 2925], [1084, 222]]","[[70688, 3919], [883, 224]]","[[63868, 9859], [1335, 652]]","[[75502, 86], [126, 0]]"


In [44]:
results_df_val.sort_values(['acc', 'recall_covid', 'prec_covid'],
                           ascending=False).dropna(axis=1, how='all')

,loss,acc,hamming,bce,acc_Pneumonia,prec_Pneumonia,recall_Pneumonia,spec_Pneumonia,roc_auc_Pneumonia,cm_Pneumonia,...,cm_Infiltration,cm_Mass,cm_Nodule,cm_Pneumothorax,cm_Consolidation,cm_Edema,cm_Emphysema,cm_Fibrosis,cm_Pleural_Thickening,cm_Hernia
0715_232631_cxr14_densenet-121_lr1e-06_os-max4_aug-0_focal_Pneumonia,0.001268,0.767313,0.011286,1.128585,0.988714,0.100000,0.008772,0.999159,0.585601,"[[10687, 9], [113, 1]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0715_232418_cxr14_densenet-121_lr1e-06_os-max4_aug-0_Pneumonia,0.604104,0.756233,0.011286,1.128585,0.988714,0.000000,0.000000,0.999252,0.559739,"[[10688, 8], [114, 0]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0714_232500_cxr14_densenet-121_lr1e-06,12.006496,0.329264,0.102187,10.218713,0.983534,0.015152,0.008772,0.993923,0.508259,"[[10631, 65], [113, 1]]",...,"[[5318, 3737], [875, 880]]","[[9427, 928], [395, 60]]","[[9419, 814], [506, 71]]","[[10055, 473], [246, 36]]","[[9817, 646], [295, 52]]","[[10489, 161], [153, 7]]","[[10446, 247], [108, 9]]","[[10599, 67], [144, 0]]","[[10121, 434], [230, 25]]","[[10786, 9], [15, 0]]"
0716_133211_cxr14_densenet-121_lr1e-06_us_aug-0_Pneumonia,4.955719,0.222715,0.057724,5.772433,0.942276,0.011494,0.052632,0.951758,0.557651,"[[10180, 516], [108, 6]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0714_232518_cxr14_densenet-121_lr1e-06,443.915100,0.220914,0.234115,23.411525,0.946253,0.016563,0.070175,0.955591,0.547665,"[[10221, 475], [106, 8]]",...,"[[812, 8243], [92, 1663]]","[[6741, 3614], [246, 209]]","[[6279, 3954], [292, 285]]","[[8372, 2156], [179, 103]]","[[8899, 1564], [236, 111]]","[[10199, 451], [127, 33]]","[[10259, 434], [105, 12]]","[[10131, 535], [128, 16]]","[[9260, 1295], [192, 63]]","[[10784, 11], [15, 0]]"


In [45]:
results_df_test.sort_values(['acc', 'recall_covid', 'prec_covid'],
                            ascending=False).dropna(axis=1, how='all')

,loss,acc,hamming,bce,acc_Pneumonia,prec_Pneumonia,recall_Pneumonia,spec_Pneumonia,roc_auc_Pneumonia,cm_Pneumonia,...,cm_Infiltration,cm_Mass,cm_Nodule,cm_Pneumothorax,cm_Consolidation,cm_Edema,cm_Emphysema,cm_Fibrosis,cm_Pleural_Thickening,cm_Hernia
0715_232631_cxr14_densenet-121_lr1e-06_os-max4_aug-0_focal_Pneumonia,2.258127,0.567916,0.022894,2.289420,0.977106,0.000000,0.000000,0.998762,0.540268,"[[25010, 31], [555, 0]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0715_232418_cxr14_densenet-121_lr1e-06_os-max4_aug-0_Pneumonia,143.362762,0.554391,0.024457,2.445695,0.975543,0.050633,0.007207,0.997005,0.568469,"[[24966, 75], [551, 4]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0714_232500_cxr14_densenet-121_lr1e-06,28.854336,0.271215,0.149541,14.954067,0.970073,0.035242,0.014414,0.991254,0.526497,"[[24822, 219], [547, 8]]",...,"[[9771, 9713], [2583, 3529]]","[[21249, 2599], [1478, 270]]","[[21687, 2286], [1417, 206]]","[[21189, 1742], [2353, 312]]","[[21105, 2676], [1505, 310]]","[[23381, 1290], [827, 98]]","[[23329, 1174], [966, 127]]","[[25035, 126], [423, 12]]","[[23237, 1216], [1059, 84]]","[[25451, 59], [86, 0]]"
0714_232518_cxr14_densenet-121_lr1e-06,797.982788,0.194041,0.294323,29.432334,0.920183,0.026115,0.073874,0.938940,0.559447,"[[23512, 1529], [514, 41]]",...,"[[1160, 18324], [247, 5865]]","[[14405, 9443], [934, 814]]","[[14220, 9753], [831, 792]]","[[16491, 6440], [1488, 1177]]","[[16469, 7312], [945, 870]]","[[22003, 2668], [671, 254]]","[[22381, 2122], [916, 177]]","[[23670, 1491], [364, 71]]","[[19780, 4673], [816, 327]]","[[25472, 38], [85, 1]]"
0716_133211_cxr14_densenet-121_lr1e-06_us_aug-0_Pneumonia,59.725624,0.124171,0.096656,9.665573,0.903344,0.023348,0.084685,0.921489,0.546432,"[[23075, 1966], [508, 47]]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
print_columns = [
#     'acc',
#     'prec_covid','prec_pneumonia','prec_normal',
#     'recall_covid','recall_pneumonia', 'recall_normal',
    # 'spec_covid','spec_pneumonia', 'spec_normal'
    'acc_Pneumonia', 'prec_Pneumonia', 'recall_Pneumonia'
]

In [52]:
# print_columns = [c for c in results_df_test.columns if c != 'loss' and 'cm' not in c]
print((results_df_train*100).to_latex(columns=print_columns,
                               float_format='%.2f',
                              ))

\begin{tabular}{lrrr}
\toprule
{} &  acc\_Pneumonia &  prec\_Pneumonia &  recall\_Pneumonia \\
\midrule
0716\_133211\_cxr14\_densenet-121\_lr1e-06\_us\_aug-0... &          92.37 &           97.83 &             86.66 \\
0715\_232418\_cxr14\_densenet-121\_lr1e-06\_os-max4\_... &          99.97 &           99.87 &            100.00 \\
0714\_232518\_cxr14\_densenet-121\_lr1e-06             &          95.20 &            2.89 &             11.55 \\
0715\_232631\_cxr14\_densenet-121\_lr1e-06\_os-max4\_... &         100.00 &           99.99 &            100.00 \\
0714\_232500\_cxr14\_densenet-121\_lr1e-06             &          98.51 &            3.31 &              1.71 \\
\bottomrule
\end{tabular}

